In [8]:
import os
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat


In [14]:
l2_dist_vec = np.random.rand(387, 2350)

In [34]:
data_path = "../../data/FID-300/"
divided_data_path = os.path.join(data_path, "divided")

label_file = os.path.join(data_path,'label_table_train.csv')
label_map = np.loadtxt(label_file,delimiter=',',dtype='int')
label_file = os.path.join(divided_data_path,'label_table_train.csv')
div_label_map = np.loadtxt(label_file,delimiter=',',dtype='int')

label_table = label_map[:, 1]

div_lhs = div_label_map[:, 0]
lhs = label_map[:, 0]

scores = np.zeros((len(label_map), 1175))

# DIstance from upper/Lower shoe features
du = l2_dist_vec.reshape(-1, int(l2_dist_vec.shape[1]/2), 2)[:, :, 0]
dl = l2_dist_vec.reshape(-1, int(l2_dist_vec.shape[1]/2), 2)[:, :, 1]

count = 0
for i, l in enumerate(lhs):
    if 2*l not in div_lhs:
        scores[i] = du[count]
        count+=1
    else:
        scores[i] = np.maximum(du[count], dl[count+1])
score_sort = scores.argsort(1)

In [37]:
print("score_sort.shape = {}".format(score_sort.shape))
print("label_table.shape = {}".format(label_table.shape))
print("label_table = {}".format(label_table))
# label_table.shape

score_sort.shape = (240, 1175)
label_table.shape = (240,)
label_table = [1044   14 1078   63   11   39 1050    3   37   37   14   14 1056 1064
    3   60    3    3   41   33   42    3   48 1069    8   31 1059   19
 1081   29   22   34   38    9    9 1059    8 1057  863   27   42 1059
 1071 1077 1064   53    1    3   45    4   20    1 1059    5    1   50
   69   29   47 1079 1072   69    7 1060   36   49   22   22   14   51
   58   24   49   16   64   64 1052 1058   50   49   20 1046 1091   49
    5   37 1049   23   71   41 1050   14 1068   66 1041 1050   13   13
 1062   56   62   48   57    7    2 1068   54 1053   21    2 1074   15
 1075   32 1047 1071 1042 1042   10   57   28   17   43   24   73   47
    5    5   11   30 1070   57   63 1080 1073   14    8 1071   37 1076
 1080   35 1055   79   39   48 1053   33   68   64   10   52   77 1080
 1047 1065 1050   58   43 1067   33   21 1040 1055   12   37 1052   57
 1085   58 1080 1083   82   50   49   65   65   73   11 1055 1049   61
   29

In [17]:
pos_array = []
for a, c in zip(score_sort, label_table):
    pos_array.append(np.where(a==c)[0][0])
    # print("match_id ,c", np.where(a==c)[0][0], c)
pos_array = np.array(pos_array)

array([0., 0., 0., ..., 0., 0., 0.])

In [4]:
divided_data_path = os.path.join(data_path, "divided")

ref_path_orig = os.path.join(data_path, "references")
ref_path = os.path.join(divided_data_path, "references")

train_path_orig = os.path.join(data_path, "tracks_cropped_train")
val_path_orig = os.path.join(data_path, "tracks_cropped_val")

train_path = os.path.join(divided_data_path, "tracks_cropped_train")
val_path = os.path.join(divided_data_path, "tracks_cropped_val")

In [5]:
if not os.path.exists(divided_data_path):
    os.makedirs(divided_data_path)

if not os.path.exists(train_path):
    os.makedirs(train_path)
    
if not os.path.exists(val_path):
    os.makedirs(val_path)
    
if not os.path.exists(ref_path):
    os.makedirs(ref_path)

In [6]:
flist = os.listdir(ref_path_orig)
flist.sort()

In [6]:
for f in flist:
    id = int(f[:5])
    src = os.path.join(ref_path_orig, f)
    I = skimage.io.imread(src)
    mid = int(I.shape[0]/2)
    I_U = I[:mid, :]
    I_B = I[mid:, :]

    dst_U = os.path.join(ref_path, "{:05d}.png".format(2*id-1))
    dst_B = os.path.join(ref_path, "{:05d}.png".format(2*id))
    skimage.io.imsave(dst_U, I_U)
    skimage.io.imsave(dst_B, I_B)

In [7]:
flist = os.listdir(train_path_orig)
flist.sort()

label_file = open(data_path + "label_table_train.csv")
label_map = np.loadtxt(label_file,delimiter=',',dtype='int')
label_dict = dict((a,b) for a,b in label_map)
label_map = np.zeros((0,2)).astype('int')

In [8]:
for f in flist:
    src = os.path.join(train_path_orig, f)
    I = skimage.io.imread(src)
    id = int(f[:5])
    
    if I.shape[0]/I.shape[1] > 2:
        mid = int(I.shape[0]/2)

        I_U = I[:mid, :]
        dst_U = os.path.join(train_path, "{:05d}.png".format(2*id-1))
        skimage.io.imsave(dst_U, I_U)

        I_B = I[mid:, :]
        dst_B = os.path.join(train_path, "{:05d}.png".format(2*id))
        skimage.io.imsave(dst_B, I_B)
        label_map = np.vstack((label_map, [2*id-1, 2*label_dict[id]-1]))
        label_map = np.vstack((label_map, [2*id, 2*label_dict[id]]))
        
    else:
        dst_U = os.path.join(train_path, "{:05d}.png".format(2*id-1))
        skimage.io.imsave(dst_U, I_U)
        label_map = np.vstack((label_map, [2*id-1, 2*label_dict[id]-1]))

In [9]:
label_path = os.path.join(divided_data_path, "label_table_train.csv")
np.savetxt(label_path, label_map, fmt="%d", delimiter=",")

In [10]:
flist = os.listdir(val_path_orig)
flist.sort()

label_file = open(data_path + "label_table_val.csv")
label_map = np.loadtxt(label_file,delimiter=',',dtype='int')
label_dict = dict((a,b) for a,b in label_map)
label_map = np.zeros((0,2)).astype('int')

In [11]:
for f in flist:
    src = os.path.join(val_path_orig, f)
    I = skimage.io.imread(src)
    id = int(f[:5])
    
    if I.shape[0]/I.shape[1] > 2:
        mid = int(I.shape[0]/2)

        I_U = I[:mid, :]
        dst_U = os.path.join(val_path, "{:05d}.png".format(2*id-1))
        skimage.io.imsave(dst_U, I_U)

        I_B = I[mid:, :]
        dst_B = os.path.join(val_path, "{:05d}.png".format(2*id))
        skimage.io.imsave(dst_B, I_B)
        label_map = np.vstack((label_map, [2*id-1, 2*label_dict[id]-1]))
        label_map = np.vstack((label_map, [2*id, 2*label_dict[id]]))
        
    else:
        dst_U = os.path.join(val_path, "{:05d}.png".format(2*id-1))
        skimage.io.imsave(dst_U, I_U)
        label_map = np.vstack((label_map, [2*id-1, 2*label_dict[id]-1]))

In [12]:
label_path = os.path.join(divided_data_path, "label_table_val.csv")
np.savetxt(label_path, label_map, fmt="%d", delimiter=",")

In [24]:
# label_file = os.path.join(divided_data_path,'label_table_train.csv')
# label_map = np.loadtxt(label_file,delimiter=',',dtype='int')

In [26]:
# label_map

In [27]:
label_file = os.path.join(data_path,'label_table_train.csv')
label_map = np.loadtxt(label_file,delimiter=',',dtype='int')

In [28]:
label_map

array([[   1, 1044],
       [   2,   14],
       [   4, 1078],
       [   5,   63],
       [   6,   11],
       [   7,   39],
       [   8, 1050],
       [   9,    3],
       [  10,   37],
       [  11,   37],
       [  12,   14],
       [  13,   14],
       [  14, 1056],
       [  15, 1064],
       [  17,    3],
       [  19,   60],
       [  20,    3],
       [  21,    3],
       [  22,   41],
       [  23,   33],
       [  24,   42],
       [  25,    3],
       [  27,   48],
       [  28, 1069],
       [  29,    8],
       [  30,   31],
       [  31, 1059],
       [  32,   19],
       [  34, 1081],
       [  36,   29],
       [  37,   22],
       [  38,   34],
       [  39,   38],
       [  41,    9],
       [  43,    9],
       [  45, 1059],
       [  46,    8],
       [  48, 1057],
       [  49,  863],
       [  53,   27],
       [  56,   42],
       [  57, 1059],
       [  58, 1071],
       [  59, 1077],
       [  62, 1064],
       [  63,   53],
       [  64,    1],
       [  66,

In [29]:
label_file = os.path.join(divided_data_path,'label_table_train.csv')
div_label_map = np.loadtxt(label_file,delimiter=',',dtype='int')

In [59]:
div_lhs = div_label_map[:, 0]
lhs = label_map[:, 0]

In [32]:
scores = np.zeros((len(div_label_map), 1175))

1044
14
1078
63
11
39
1050
3
37
37
14
14
1056
1064
3
60
3
3
41
33
42
3
48
1069
8
31
1059
19
1081
29
22
34
38
9
9
1059
8
1057
863
27
42
1059
1071
1077
1064
53
1
3
45
4
20
1
1059
5
1
50
69
29
47
1079
1072
69
7
1060
36
49
22
22
14
51
58
24
49
16
64
64
1052
1058
50
49
20
1046
1091
49
5
37
1049
23
71
41
1050
14
1068
66
1041
1050
13
13
1062
56
62
48
57
7
2
1068
54
1053
21
2
1074
15
1075
32
1047
1071
1042
1042
10
57
28
17
43
24
73
47
5
5
11
30
1070
57
63
1080
1073
14
8
1071
37
1076
1080
35
1055
79
39
48
1053
33
68
64
10
52
77
1080
1047
1065
1050
58
43
1067
33
21
1040
1055
12
37
1052
57
1085
58
1080
1083
82
50
49
65
65
73
11
1055
1049
61
29
32
83
27
22
22
21
1055
69
1087
1087
1053
45
12
21
34
1088
1039
1040
1089
68
11
1096
30
30
1044
1094
1085
56
17
4
56
56
49
51
1044
69
17
37
88
20
43
1071
60
60
44
1049
79
88
1055
1044
60
74
1096
1041
1082
5
46


In [38]:
div_label_map.shape

(387, 2)

In [44]:
l2_dist_vec = np.zeros((387, 2350))

In [53]:
# DIstance from upper vectors
du = l2_dist_vec.reshape(-1, int(l2_dist_vec.shape[1]/2), 2)[:, :, 0]
dl = l2_dist_vec.reshape(-1, int(l2_dist_vec.shape[1]/2), 2)[:, :, 1]

In [56]:
np.maximum(np.array([1,2,3]), np.array([1,3,2]))

array([1, 3, 3])

In [67]:
print(lhs)

[  1   2   4   5   6   7   8   9  10  11  12  13  14  15  17  19  20  21
  22  23  24  25  27  28  29  30  31  32  34  36  37  38  39  41  43  45
  46  48  49  53  56  57  58  59  62  63  64  66  67  68  69  70  71  72
  73  74  76  77  78  79  80  81  82  83  84  87  89  90  91  92  93  94
  95  97  98  99 100 101 103 104 105 107 108 109 110 111 113 114 115 116
 117 118 119 121 122 123 124 125 127 128 129 130 131 132 133 136 137 138
 139 140 141 142 143 144 145 146 147 148 149 150 151 156 159 161 162 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 200 201
 202 203 204 207 208 209 210 211 212 213 214 215 216 217 219 220 221 222
 223 225 226 227 228 229 230 231 232 233 234 235 237 238 240 241 242 244
 245 246 247 254 255 256 257 259 260 261 262 263 264 265 267 268 269 271
 272 273 274 275 276 277 278 279 280 282 283 284 285 286 287 288 289 291
 292 293 297 298 299 300]


In [70]:
print(div_lhs)

[  1   3   4   7   8   9  10  11  12  13  15  17  18  19  21  22  23  24
  25  27  29  33  34  37  39  40  41  43  44  45  47  49  50  53  54  55
  56  57  59  60  61  62  63  67  71  72  73  74  75  76  77  78  81  85
  89  90  91  92  95  96  97  98 105 106 111 112 113 115 116 117 118 123
 125 126 127 131 133 134 135 136 137 138 139 140 141 143 144 145 147 148
 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 173 174 177 178 179 180 181 183 184 185 186 187 188 189 190 193 194 195
 197 199 200 201 205 206 207 208 209 210 213 214 215 216 217 219 220 221
 222 225 226 227 228 229 230 231 232 233 234 235 237 238 241 243 244 245
 247 248 249 250 253 254 255 257 258 259 261 262 263 264 265 266 271 272
 273 274 275 277 278 279 281 282 283 284 285 286 287 288 289 291 292 293
 295 296 297 299 301 311 312 317 318 321 322 323 324 327 328 329 330 331
 332 333 334 335 336 337 339 340 341 342 343 344 345 347 349 350 351 353
 355 356 357 358 359 360 361 363 364 365 366 367 36

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

layer_id = 5
model = models.resnet50(pretrained=True)
modules = list(model.children())[:layer_id] 
net_base = nn.Sequential(*modules)

In [10]:
list(model.children())[:6]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [26]:
for param in net_base[-1][-1].parameters():
    print(param)

Parameter containing:
tensor([[[[ 5.7859e-03]],

         [[ 1.3108e-03]],

         [[-4.1067e-03]],

         ...,

         [[-4.2579e-03]],

         [[-4.1070e-02]],

         [[-3.8527e-02]]],


        [[[ 1.9597e-03]],

         [[-1.0637e-02]],

         [[ 1.8890e-02]],

         ...,

         [[ 4.8657e-03]],

         [[-1.1662e-01]],

         [[ 7.1150e-02]]],


        [[[-1.3912e-02]],

         [[ 1.8223e-03]],

         [[ 2.2519e-02]],

         ...,

         [[-2.2903e-02]],

         [[-1.1148e-02]],

         [[ 7.2517e-03]]],


        ...,


        [[[ 2.5342e-03]],

         [[-3.2014e-03]],

         [[-1.0234e-02]],

         ...,

         [[ 1.2434e-02]],

         [[-1.1588e-02]],

         [[ 1.7599e-02]]],


        [[[ 4.4654e-04]],

         [[ 5.1545e-03]],

         [[-2.1940e-02]],

         ...,

         [[-1.4156e-02]],

         [[ 6.5771e-02]],

         [[-2.3287e-02]]],


        [[[-2.4907e-03]],

         [[-7.4796e-05]],

         [[-3.

In [8]:
import numpy as np
a = np.random.rand(2,3,4)

In [9]:
print(a.shape)
a

(2, 3, 4)


array([[[0.49127409, 0.59324154, 0.43689013, 0.56582177],
        [0.47537999, 0.06962565, 0.0249453 , 0.08658383],
        [0.34935651, 0.44845262, 0.90303455, 0.50780448]],

       [[0.12280073, 0.54399206, 0.03181867, 0.13016012],
        [0.77691304, 0.58415058, 0.40649186, 0.16595774],
        [0.90659031, 0.68229594, 0.99908489, 0.48462859]]])

In [14]:
print(a.mean(2).shape)
a-a.mean(2)[:,:,None]

(2, 3)


array([[[-0.03053279,  0.07143466, -0.08491675,  0.04401489],
        [ 0.3112463 , -0.09450804, -0.13918839, -0.07754986],
        [-0.20280553, -0.10370942,  0.35087251, -0.04435756]],

       [[-0.08439216,  0.33679917, -0.17537423, -0.07703278],
        [ 0.29353474,  0.10077228, -0.07688644, -0.31742057],
        [ 0.13844038, -0.08585399,  0.23093495, -0.28352134]]])

In [18]:
import torch
import numpy as np
a = torch.tensor(np.zeros((2,3,4,5)))

In [19]:
torch.norm(a, dim=1)

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], dtype=torch.float64)

In [32]:
v1 = torch.tensor(np.zeros((2,3,4)))
v2 = torch.tensor(np.zeros((2,3,4)))

In [33]:
den = torch.norm(v1, dim=2)*torch.norm(v2, dim=2) + 1e-12

In [35]:
num = (v1*v2).sum(axis=2)

In [34]:
den.shape

torch.Size([2, 3])

In [36]:
num.shape

torch.Size([2, 3])

In [29]:
v1 = np.zeros((2,3,4))
v2 = np.zeros((2,3,4))
den = np.linalg.norm(v1, axis=1)*np.linalg.norm(v2, axis=1) + 1e-12

In [30]:
den.shape

(2, 4)

In [37]:
a.shape

torch.Size([2, 3, 4, 5])

In [41]:
a.numel()

120

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [10]:
model = models.resnet50()

In [12]:
modules = list(model.children())[:7] 
net_base = nn.Sequential(*modules)
for param in net_base[-1][-1].parameters():
    a  = param

In [15]:
a

Parameter containing:
tensor([0., 0., 0.,  ..., 0., 0., 0.], requires_grad=True)

In [33]:
for layer in list(net_base[-1][-1].children())[-3:-1]:
    layer.weight.requires_grad = True